### Q1. What is Gradient Boosting Regression?

Gradient Boosting Regression is an ensemble learning technique used for regression tasks. It involves sequentially adding weak learners (usually decision trees) to an ensemble, where each new tree corrects errors made by the previous ones. It minimizes the loss function by using gradient descent optimization, fitting each new tree to the residual errors of the previous model.

### Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple regression problem as an example and train the model on a small dataset. Evaluate the model's performance using metrics such as mean squared error and R-squared.

In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score

# Generate a small dataset for regression
X, y = make_regression(n_samples=100, n_features=1, noise=5, random_state=42)

# Define the base learner (Decision Tree Regressor)
class DecisionTree:
    def __init__(self, max_depth=1):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.split_feature = None
        self.split_value = None
        self.left = None
        self.right = None
        self.prediction = np.mean(y)

        if self.max_depth > 0:
            best_sse = np.inf

            for feature in range(X.shape[1]):
                feature_values = np.unique(X[:, feature])
                for value in feature_values:
                    left_indices = X[:, feature] <= value
                    right_indices = X[:, feature] > value

                    y_left = y[left_indices]
                    y_right = y[right_indices]

                    if len(y_left) == 0 or len(y_right) == 0:
                        continue

                    sse = np.sum((y_left - np.mean(y_left))**2) + np.sum((y_right - np.mean(y_right))**2)
                    if sse < best_sse:
                        best_sse = sse
                        self.split_feature = feature
                        self.split_value = value

            if self.split_feature is not None:
                left_indices = X[:, self.split_feature] <= self.split_value
                right_indices = X[:, self.split_feature] > self.split_value

                self.left = DecisionTree(max_depth=self.max_depth - 1)
                self.left.fit(X[left_indices], y[left_indices])

                self.right = DecisionTree(max_depth=self.max_depth - 1)
                self.right.fit(X[right_indices], y[right_indices])

    def predict(self, X):
        if self.split_feature is None:
            return self.prediction
        else:
            predictions = np.zeros(len(X))
            left_indices = X[:, self.split_feature] <= self.split_value
            right_indices = X[:, self.split_feature] > self.split_value

            predictions[left_indices] = self.left.predict(X[left_indices])
            predictions[right_indices] = self.right.predict(X[right_indices])

            return predictions


# Define the Gradient Boosting Regressor
class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        y_pred = np.full(len(y), np.mean(y))

        for _ in range(self.n_estimators):
            residual = y - y_pred
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X, residual)
            y_pred += self.learning_rate * tree.predict(X)
            self.estimators.append(tree)

    def predict(self, X):
        y_pred = np.zeros(len(X))
        for tree in self.estimators:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred


# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Initialize and train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_regressor.fit(X_train, y_train)

# Make predictions
y_pred = gb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 55.63481013612932
R-squared: 0.9622905041637521


### Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to optimise the performance of the model. Use grid search or random search to find the best hyperparameters

In [7]:
from sklearn.base import BaseEstimator, RegressorMixin

class GradientBoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        y_pred = np.full(len(y), np.mean(y))

        for _ in range(self.n_estimators):
            residual = y - y_pred
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X, residual)
            y_pred += self.learning_rate * tree.predict(X)
            self.estimators.append(tree)

    def predict(self, X):
        y_pred = np.zeros(len(X))
        for tree in self.estimators:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'learning_rate': self.learning_rate,
            'max_depth': self.max_depth
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self


In [6]:
# Your dataset split into X_train, X_test, y_train, y_test

from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearch
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [1, 2, 3]
}

# Initialize the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor()

# Create GridSearchCV instance
grid_search = GridSearchCV(gb_regressor, param_grid, cv=3, scoring='neg_mean_squared_error')

# Fit the grid search to find the best parameters
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters:", best_params)

# Use the best estimator to make predictions
y_pred = best_estimator.predict(X_test)

# Evaluate the model with the best parameters
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Best Parameters: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 100}
Mean Squared Error: 52.373232214161746
R-squared: 0.9645012146661712



### Q4. What is a weak learner in Gradient Boosting?

**A weak learner** in the context of Gradient Boosting is a model that's only slightly better than random guessing on a given problem. It's a base model that performs just a bit better than chance. In Gradient Boosting, decision trees are commonly used as weak learners. These trees are often shallow (low depth) to prevent overfitting and make them weak enough to focus on the residual errors of the previous models.

### Q5. What is the intuition behind the Gradient Boosting algorithm?

**The intuition** behind Gradient Boosting involves creating a strong model by iteratively improving upon the errors of previous models. It works in a stepwise manner, where each subsequent model corrects the errors made by the previous ones. By combining weak learners sequentially, it constructs a powerful ensemble that can fit complex patterns in the data.

### Q6. How does the Gradient Boosting algorithm build an ensemble of weak learners?

**Gradient Boosting** builds an ensemble by sequentially fitting new models to the residual errors made by the previous models. Here's a simplified outline of the process:

1. **Initialize with a simple model:** The algorithm starts with a simple model that predicts the mean (or another constant) for the target variable.
2. **Fit a weak learner (typically a decision tree):** It then fits a weak learner to the residuals (errors) of the initial model. This weak learner focuses on correcting the errors made by the initial model.
3. **Iteratively build subsequent models:** It continues this process, where each new model focuses on the errors that the ensemble of previous models has made, gradually reducing the overall error.

### Q7. What are the steps involved in constructing the mathematical intuition of the Gradient Boosting algorithm?

**Mathematically**, the Gradient Boosting algorithm can be understood in steps:

1. **Initialize with a constant:** Start by predicting the mean (or another simple value) for the target variable.
2. **Compute residuals:** Calculate the residuals (errors) by subtracting the predicted values from the actual target values.
3. **Fit a weak learner to residuals:** Train a weak learner (usually a decision tree) on these residuals to predict these errors.
4. **Update predictions:** Combine the predictions of the weak learner with the previous predictions. This update is performed with a learning rate, controlling the contribution of each model to the ensemble.
5. **Repeat steps 2-4:** Iterate this process, focusing each new model on the errors that the ensemble of previous models has made, thereby reducing the overall error.

These steps, when performed iteratively, gradually improve the overall model by minimizing the residuals and learning from the mistakes made by the previous models.

Understanding these steps mathematically helps in grasping how each subsequent model is fitted to the errors of the previous model, gradually improving the predictive capability of the ensemble.